# Tutorial 

https://techcommunity.microsoft.com/t5/azure-database-for-postgresql/analyzing-gps-trajectories-at-scale-with-postgres-mobilitydb-amp/ba-p/1859278


# Launch docker

docker pull mobilitydb/mobilitydb  
docker volume create mobilitydb_data  
docker run --name "mobilitydb" -d -p 25432:5432 -v mobilitydb_data:/var/lib/postgresql mobilitydb/mobilitydb  


In [1]:
# ! pip install psycopg2-binary

     |████████████████████████████████| 3.0MB 7.3MB/s eta 0:00:01


In [ ]:
# https://coderedirect.com/questions/349555/docker-postgres-and-pgadmin-4-connection-refused
# docker network ls
# docker network inspect 123254035bc7

In [2]:
import pandas as pd
import psycopg2

In [3]:
def execute_query(query, fetch = False):
    conn = psycopg2.connect("dbname='mobilitydb' user='docker' host='172.17.0.3' password='docker'") 
    conn.autocommit = True
    cursor = conn.cursor()
    cursor.execute(query)
    if fetch:
        result = cursor.fetchall()
    else:
        result = None
    conn.close()
    return result

# Load extensions

In [6]:
create_extensions = """
CREATE EXTENSION PostGIS;
CREATE EXTENSION MobilityDB CASCADE;
"""
execute_query(create_extensions)

DuplicateObject: extension "postgis" already exists


# Create new tables

In [7]:
create_gps_point_table = """
DROP TABLE if exists gpsPoint CASCADE;
CREATE TABLE gpsPoint (tripID int, pointID int, t timestamp, geom geometry(Point, 3812));
"""

In [8]:
create_billboard_table = """
DROP TABLE if exists billboard CASCADE;
CREATE TABLE billboard(billboardID int, geom geometry(Point, 3812));
"""

# Insert generated data into tables

In [9]:
def gps_point_insert_query(n):
    s = "\n".join([
        f"""
    ({i*2+1}, 1, '2020-04-21 08:37:27', 'SRID=3812;POINT({651096+i*200}.993815166 667028.114604598)'),
    ({i*2+1}, 2, '2020-04-21 08:37:39', 'SRID=3812;POINT({651080+i*200}.424535144 667123.352304597)'),    
    ({i*2+1}, 3, '2020-04-21 08:38:06', 'SRID=3812;POINT({651067+i*200}.607438095 667173.570340437)'),
    ({i*2+1}, 4, '2020-04-21 08:38:31', 'SRID=3812;POINT({651052+i*200}.741845233 667213.026797244)'),    
    ({i*2+1}, 5, '2020-04-21 08:38:49', 'SRID=3812;POINT({651029+i*200}.676773636 667255.556944161)'),    
    ({i*2+1}, 6, '2020-04-21 08:39:08', 'SRID=3812;POINT({651018+i*200}.401101238 667271.441380755)'),    
    ({i*2+2}, 1, '2020-04-21 08:39:29', 'SRID=3812;POINT({651262+i*200}.17004873  667119.331513367)'),    
    ({i*2+2}, 2, '2020-04-21 08:38:36', 'SRID=3812;POINT({651201+i*200}.431447782 667089.682115196)'),    
    ({i*2+2}, 3, '2020-04-21 08:38:43', 'SRID=3812;POINT({651186+i*200}.853162155 667091.138189286)'),    
    ({i*2+2}, 4, '2020-04-21 08:38:49', 'SRID=3812;POINT({651181+i*200}.995412783 667077.531372716)'),    
    ({i*2+2}, 5, '2020-04-21 08:38:56', 'SRID=3812;POINT({651101+i*200}.820139904 667041.076539663)'),"""
        for i in range(n//2)
    ])
    start = "INSERT INTO gpsPoint Values"
    query = f"{start} {s[:-1]};"
    return query

print(gps_point_insert_query(2))

INSERT INTO gpsPoint Values 
    (1, 1, '2020-04-21 08:37:27', 'SRID=3812;POINT(651096.993815166 667028.114604598)'),
    (1, 2, '2020-04-21 08:37:39', 'SRID=3812;POINT(651080.424535144 667123.352304597)'),    
    (1, 3, '2020-04-21 08:38:06', 'SRID=3812;POINT(651067.607438095 667173.570340437)'),
    (1, 4, '2020-04-21 08:38:31', 'SRID=3812;POINT(651052.741845233 667213.026797244)'),    
    (1, 5, '2020-04-21 08:38:49', 'SRID=3812;POINT(651029.676773636 667255.556944161)'),    
    (1, 6, '2020-04-21 08:39:08', 'SRID=3812;POINT(651018.401101238 667271.441380755)'),    
    (2, 1, '2020-04-21 08:39:29', 'SRID=3812;POINT(651262.17004873  667119.331513367)'),    
    (2, 2, '2020-04-21 08:38:36', 'SRID=3812;POINT(651201.431447782 667089.682115196)'),    
    (2, 3, '2020-04-21 08:38:43', 'SRID=3812;POINT(651186.853162155 667091.138189286)'),    
    (2, 4, '2020-04-21 08:38:49', 'SRID=3812;POINT(651181.995412783 667077.531372716)'),    
    (2, 5, '2020-04-21 08:38:56', 'SRID=3812;POIN

In [10]:
def billboard_insert_query(n):
    s = "\n".join([
        f"""
        ({i*2+1}, 'SRID=3812;POINT({651066 + i*200}.289442793 667213.589577551)'),
        ({i*2+2}, 'SRID=3812;POINT({651110 + i* 200}.505092035 667166.698041233)'),"""
        for i in range(n // 2)
    ])
    start = "INSERT INTO billboard Values"
    query = f"{start} {s[:-1]};"
    return query
print(billboard_insert_query(2))

INSERT INTO billboard Values 
        (1, 'SRID=3812;POINT(651066.289442793 667213.589577551)'),
        (2, 'SRID=3812;POINT(651110.505092035 667166.698041233)');


# Post-GIS queries

In [11]:
create_post_gis_view = """
DROP VIEW if exists timeGPS;
CREATE VIEW timeGPS AS
WITH pointPair AS(
     SELECT tripID, pointID AS p1, t AS t1, geom AS geom1,
       lead(pointID, 1) OVER (PARTITION BY tripID ORDER BY pointID) p2,
       lead(t, 1) OVER (PARTITION BY tripID ORDER BY pointID) t2,
       lead(geom, 1) OVER (PARTITION BY tripID ORDER BY pointID) geom2    
     FROM gpsPoint
   ), segment AS(
     SELECT tripID, p1, p2, t1, t2,
       st_makeline(geom1, geom2) geom
    FROM pointPair
    WHERE p2 IS NOT NULL    
  ), approach AS(
    SELECT tripID, p1, p2, t1, t2, a.geom,
      st_intersection(a.geom, st_exteriorRing(st_buffer(b.geom, 30))) visibilityTogglePoint
    FROM segment a, billboard b
    WHERE st_dwithin(a.geom, b.geom, 30)
  )
  SELECT tripID, p1, p2, t1, t2, geom, visibilityTogglePoint,
    (st_lineLocatePoint(geom, visibilityTogglePoint) * (t2 - t1)) + t1 visibilityToggleTime,
	st_lineLocatePoint(geom, visibilityTogglePoint) * (t2 - t1) visibilityTime
  FROM approach;
"""

postgis_time = """
explain analyze select SUM(visibilityTime) FROM timeGPS;
"""

# Mobility Queries

In [18]:
create_mobility_bus_table = """
DROP table if exists busTrip CASCADE;
CREATE TABLE busTrip(tripID, trip) AS
  SELECT tripID,tgeompoint_seq(array_agg(tgeompoint_inst(geom, t) ORDER BY t))
FROM gpsPoint
GROUP BY tripID;
"""
create_mobility_view = """
DROP VIEW if exists mob;
CREATE view  mob as 
SELECT astext(atperiodset(trip, getTime(atValue(tdwithin(a.trip, b.geom, 30), TRUE))))
FROM busTrip a, billboard b
WHERE dwithin(a.trip, b.geom, 30);
"""

mobility_time = """
explain analyze select * from mob;
"""

# Put it all together

In [19]:
df = pd.DataFrame(columns = ["np", "nb", "time_gis", "time_mobility"])

In [ ]:
for n in [4, 40, 400, 4000, 40000]:
    for _ in range(5):
        execute_query(create_gps_point_table)
        execute_query(create_billboard_table)

        execute_query(gps_point_insert_query(n))
        execute_query(billboard_insert_query(n))

        execute_query(create_post_gis_view)
        res = execute_query(postgis_time, fetch = True)
        time_gis = res[-1][0].split(":")[1].strip()
        print(n, time_gis)

        execute_query(create_mobility_bus_table)
        execute_query(create_mobility_view)
        res = execute_query(mobility_time, fetch = True)
        time_mob = res[-1][0].split(":")[1].strip()
        print(n, time_mob)

        df.loc[df.shape[0]] = [11*n//2, n, time_gis, time_mob]
        df.to_pickle("results.pkl")

4 281.386 ms
4 37.716 ms
4 277.190 ms
4 39.764 ms
4 282.596 ms
4 37.926 ms
4 291.498 ms
4 41.114 ms
4 271.508 ms
4 37.255 ms
40 281.189 ms
40 42.697 ms
40 296.598 ms
40 40.697 ms
40 294.929 ms
40 41.328 ms
40 276.900 ms
40 40.614 ms
40 284.580 ms
40 40.297 ms
400 823.466 ms
400 251.967 ms
400 796.670 ms
400 260.733 ms
400 814.660 ms
400 252.603 ms
400 822.559 ms
400 248.112 ms
400 808.634 ms
400 244.115 ms
4000 53546.834 ms
4000 9245.564 ms
4000 52924.564 ms
4000 8935.010 ms
4000 53176.031 ms
4000 9164.303 ms
4000 53563.699 ms
4000 9100.357 ms
4000 52940.702 ms
4000 9148.478 ms


# Debug

In [ ]:
execute_query(create_gps_point_table)
execute_query(create_billboard_table)

execute_query("select * from gpsPoint", fetch = True)

execute_query("delete from gpsPoint")

In [ ]:
len(execute_query("select * from gpsPoint;", fetch = True))

len(execute_query("select * from billboard;", fetch = True))